In [1]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=192aa1fd2c76e3815ea18c09a2307589f67bfffcadef0918508c8fdd29ad65e6
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from bayes_opt import BayesianOptimization

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
#         print(os.path.join(dirname, filename))
        #pass

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
all_train_data = pd.read_csv('drive/MyDrive/People Analytics/train.csv')
all_test_data = pd.read_csv('drive/MyDrive/People Analytics/test.csv')
sample_submission = pd.read_csv('drive/MyDrive/People Analytics/sample_submission.csv')
dropped_columns = ['year_graduated']
y_train = all_train_data['Best Performance']
X_train = all_train_data.drop('Best Performance',axis=1)

In [5]:
#y_train

In [5]:
for col in dropped_columns:
    X_train = X_train.drop(col,axis=1)
X_test = all_test_data
for col in dropped_columns:
    X_test = X_test.drop(col,axis=1)
X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

In [7]:
#np.unique(X_test[cat]) #broken code

In [9]:
#cat for categorical
cat_columns = ['job_level', 'person_level', 'Employee_type', 'Employee_status', 'gender', 'marital_status_maried(Y/N)', 'Education_level', 'achievement_target_1', 'achievement_target_2', 'achievement_target_3']
for cat in cat_columns:
    labelencoder = LabelEncoder()
    X_train_[cat] = labelencoder.fit_transform(X_train_[cat].astype(str))
    X_test_[cat] = labelencoder.transform(X_test_[cat].astype(str))
for cat in cat_columns:
    labelencoder = LabelEncoder()
    X_train[cat] = labelencoder.fit_transform(X_train[cat].astype(str))
    X_test[cat] = labelencoder.transform(X_test[cat].astype(str))  
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


ADVERSARIAL VALIDATION

In [ ]:
#XGB

In [10]:
from xgboost.sklearn import XGBClassifier
import xgboost as xgb

In [11]:
params = {'n_estimators':100,
          'learning_rate':0.01
          }
model_xgb = xgb.XGBClassifier(**params)
model_xgb.fit(X_train,y_train)
model_xgb.predict_proba(X_test)[:,1]
sample_submission['Best Performance'] = model_xgb.predict_proba(X_test)[:,1]
sample_submission.to_csv('submission.csv')

In [24]:
class xgbm_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        params = {'colsample_bytree':1.0, 
                  'learning_rate':0.01,
                  'n_estimators':100, 
                  'random_state':0, 
                  'reg_alpha':0.0, 
                  'reg_lambda':0.0, 
                  'subsample':1.0,
                  'gamma':0.0
                  }
        params['max_depth'] = int(param['max_depth'])
        params['learning_rate'] = 0.01

        return params
        
    def evaluate(self, max_depth,gamma,n_estimators,learning_rate,reg_alpha,reg_lambda,min_child_weight,colsample_bytree):
        params = {'max_depth':max_depth,
                  'n_estimators': int(n_estimators),
                  'subsample': 0.8,
                  'learning_rate':learning_rate,
                  'gamma': gamma,
                  'reg_alpha':reg_alpha,
                  'reg_lambda':reg_lambda,
                  'min_child_weight':min_child_weight,
                  'colsample_bytree':colsample_bytree
                  }
        #params['max_depth'] = int(param['max_depth'])
        params = self.clean_param(params)

        xgbm_model = xgb.XGBClassifier(**params)
        xgbm_model.fit(self.x_train, self.y_train)
        y_pred = xgbm_model.predict_proba(self.x_test)
        predictions = y_pred[:,1]
#         rmse = np.sqrt(mean_squared_error(self.y_test, predictions))
#         return -1*rmse
        acc = roc_auc_score(self.y_test,predictions)
        return acc

lt = xgbm_target(X_train_, y_train_, X_test_, y_test_)
xgbmBO = BayesianOptimization(lt.evaluate, {
                                            'max_depth': (1, 10),
                                            'gamma': (0, 0.5),
                                            'learning_rate':(0.005,0.01),
                                            'n_estimators':(100,2000),
                                            'reg_alpha':(0,10),
                                            'reg_lambda':(10,80),
                                            'min_child_weight':(1,6),
                                            'colsample_bytree':(0.5,1)
                                           }, 
                             random_state=42)

xgbmBO.maximize(n_iter=5, init_points=10, acq='ei')

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7323   |  0.6873   |  0.4754   |  0.00866  |  6.388    |  1.78     |  396.4    |  0.5808   |  70.63    |
|  2        |  0.7303   |  0.8006   |  0.354    |  0.005103 |  9.729    |  5.162    |  503.4    |  1.818    |  22.84    |
|  3        |  0.7327   |  0.6521   |  0.2624   |  0.00716  |  3.621    |  4.059    |  365.0    |  2.921    |  35.65    |
|  4        |  0.734    |  0.728    |  0.3926   |  0.005998 |  5.628    |  3.962    |  188.3    |  6.075    |  21.94    |
|  5        |  0.7332   |  0.5325   |  0.4744   |  0.009828 |  8.276    |  2.523    |  285.6    |  6.842    |  40.81    |
|  6        |  0.7303   |  0.561    |  0.2476   |  0.005172 |  9.184    |  2.294    |  1.359e+0 |  3.117    |  46.4     |
|  7        |  0.7303   

In [25]:
y_label_adv = np.zeros((X_train.shape[0]+ X_test.shape[0]))
y_label_adv[:X_train.shape[0]] = 1
adversarial_data = pd.concat((X_train,X_test))
model_xgb = xgb.XGBClassifier()
X_train_, X_test_, y_train_, y_test_ = train_test_split(adversarial_data, y_label_adv, test_size=0.3, random_state=42)

model_xgb.fit(X_train_,y_train_)
y_pred  = model_xgb.predict(X_test_)
roc_auc_score(y_pred,y_test_)

0.8933928571428571

In [27]:
params = xgbmBO.max['params']
print(params)

{'colsample_bytree': 0.728034992108518, 'gamma': 0.3925879806965068, 'learning_rate': 0.005998368910791798, 'max_depth': 5.628109945722504, 'min_child_weight': 3.9620728443102124, 'n_estimators': 188.25578416799567, 'reg_alpha': 6.075448519014383, 'reg_lambda': 21.936688658110405}


In [28]:
params['max_depth']= int(params['max_depth'])
params['n_estimators']= int(params['n_estimators'])

In [29]:
classifier2 = XGBClassifier(**params).fit(X_train,y_train)

In [35]:
train_p2 = classifier2.predict(X_train)

In [37]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(train_p2, y_train))

              precision    recall  f1-score   support

           0       1.00      0.88      0.94     22005
           1       0.00      0.00      0.00         0

    accuracy                           0.88     22005
   macro avg       0.50      0.44      0.47     22005
weighted avg       1.00      0.88      0.94     22005



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
cm = confusion_matrix(train_p2, y_train)
acc = cm.diagonal().sum()/cm.sum()
print(acc)

0.87875482844808
